# Utils

In [1]:
import scipy.sparse as sp
import numpy as np, os, re, itertools, math
import tensorflow as tf
from abc import abstractmethod
import torch
import tensorflow.compat.v1 as tf
#import tensorflow as tf
import sys
import time
import pandas as pd

def build_knowledge(training_instances, validate_instances, testing_instances):
    """
    testing_instances: 為了建立所有項目的字典, 因此加入 testing_instances 以得到資料集的所有項目, 但項目出現次數不計算在鄰接矩陣裡。
    """
    MAX_SEQ_LENGTH = 0
    item_freq_dict = {}

    for line in training_instances:
        elements = line.split("|")

        if len(elements) > MAX_SEQ_LENGTH:
            MAX_SEQ_LENGTH = len(elements)

        basket_seq = elements[:]

        for basket in basket_seq:
            item_list = re.split('[\\s]+', basket)
            for item_obs in item_list:
                if item_obs=='':
                    continue
                if item_obs not in item_freq_dict:
                    item_freq_dict[item_obs] = 1
                else:
                    item_freq_dict[item_obs] += 1

    for line in validate_instances:
        elements = line.split("|")
        
        if len(elements) > MAX_SEQ_LENGTH:
            MAX_SEQ_LENGTH = len(elements)
        
        basket_seq = elements[:]

        for basket in basket_seq:
            item_list = re.split('[\\s]+', basket)
            for item_obs in item_list:
                if item_obs=='':
                    continue
                if item_obs not in item_freq_dict:
                    item_freq_dict[item_obs] = 1
                else:
                    item_freq_dict[item_obs] += 1

    for line in testing_instances:
        elements = line.split("|")
        basket_seq = elements[:]
        for basket in basket_seq:
            item_list = re.split('[\\s]+', basket)
            for item_obs in item_list:
                if item_obs=='':
                    continue
                if item_obs not in item_freq_dict:
                    item_freq_dict[item_obs] = 0

    items = sorted(list(item_freq_dict.keys()))
    item_dict = dict()
    item_probs = []
    for item in items:
        item_dict[item] = len(item_dict)
        item_probs.append(item_freq_dict[item])

    item_probs = np.asarray(item_probs, dtype=np.float32)
    item_probs /= np.sum(item_probs)

    reversed_item_dict = dict(zip(item_dict.values(), item_dict.keys()))
    return MAX_SEQ_LENGTH, item_dict, reversed_item_dict, item_probs

def build_sparse_adjacency_matrix_v2(training_instances, validate_instances, item_dict):
    NB_ITEMS = len(item_dict)

    pairs = {}
    for line in training_instances:
        elements = line.split("|")

        basket_seq = elements[:]

        for basket in basket_seq:
            item_list = re.split('[\\s]+', basket)
            id_list = [item_dict[item] for item in item_list]

            for t in list(itertools.product(id_list, id_list)):
                add_tuple(t, pairs)

    for line in validate_instances:
        elements = line.split("|")

        basket_seq = elements[:]

        for basket in basket_seq:
            item_list = re.split('[\\s]+', basket)
            id_list = [item_dict[item] for item in item_list]

            for t in list(itertools.product(id_list, id_list)):
                add_tuple(t, pairs)

    return create_sparse_matrix(pairs, NB_ITEMS)

def add_tuple(t, pairs):
    assert len(t) == 2
    if t[0] != t[1]:
        if t not in pairs:
            pairs[t] = 1
        else:
            pairs[t] += 1

def create_sparse_matrix(pairs, NB_ITEMS):
    row = [p[0] for p in pairs]
    col = [p[1] for p in pairs]
    data = [pairs[p] for p in pairs]

    adj_matrix = sp.csc_matrix((data, (row, col)), shape=(NB_ITEMS, NB_ITEMS), dtype="float32")
    nb_nonzero = len(pairs)
    density = nb_nonzero * 1.0 / NB_ITEMS / NB_ITEMS
    print("Density: {:.6f}".format(density))

    return sp.csc_matrix(adj_matrix, dtype="float32")

def seq_batch_generator(raw_lines, item_dict, batch_size=32, is_train=True):
    total_batches = compute_total_batches(len(raw_lines), batch_size)
    
    O = []
    S = []
    L = []
    Y = []

    batch_id = 0
    while 1:
        lines = raw_lines[:]

        if is_train:
            np.random.shuffle(lines)

        for line in lines:
            elements = line.split("|")

            #label = float(elements[0])
            bseq = elements[1:-1]
            tbasket = elements[-1]

            # Keep the length for dynamic_rnn
            L.append(len(bseq))

            # Keep the original last basket
            O.append(tbasket)

            # Add the target basket
            target_item_list = re.split('[\\s]+', tbasket)
            
            Y.append(create_binary_vector(target_item_list, item_dict))

            s = []
            for basket in bseq:
                item_list = re.split('[\\s]+', basket)
                id_list = [item_dict[item] for item in item_list]
                s.append(id_list)
            S.append(s)

            if len(S) % batch_size == 0:
                yield batch_id, {'S': np.asarray(S, dtype=object), 'L': np.asarray(L), 'Y': np.asarray(Y), 'O': np.asarray(O)}
                S = []
                L = []
                O = []
                Y = []
                batch_id += 1

            if batch_id == total_batches:
                batch_id = 0
                if not is_train:
                    break

def create_binary_vector(item_list, item_dict): # 傳入每個user的target項目ID串列，轉成二元向量
    v = np.zeros(len(item_dict), dtype='int32')
    for item in item_list:
        v[item_dict[item]] = 1
    return v


def list_directory(dir, dir_only=False):
    rtn_list = []
    for f in os.listdir(dir):
        if dir_only and os.path.isdir(os.path.join(dir, f)):
            rtn_list.append(f)
        elif not dir_only and os.path.isfile(os.path.join(dir, f)):
            rtn_list.append(f)
    return rtn_list


def create_folder(dir):
    try:
        os.makedirs(dir)
    except OSError:
        pass


def read_file_as_lines(file_path):
    with open(file_path, "r") as f:
        lines = [line.rstrip('\n') for line in f]
        return lines


def recent_model_dir(dir):
    folderList = list_directory(dir, True)
    folderList = sorted(folderList, key=get_epoch)
    return folderList[-1]


def get_epoch(x):
    idx = x.index('_') + 1
    return int(x[idx:])


def compute_total_batches(nb_intances, batch_size):
    total_batches = int(nb_intances / batch_size)
    if nb_intances % batch_size != 0:
        total_batches += 1
    return total_batches


def create_identity_matrix(nb_items):
    return sp.identity(nb_items, dtype="float32").tocsr()

def create_zero_matrix(nb_items):
    return sp.csr_matrix((nb_items, nb_items), dtype="float32")

def normalize_adj(adj_matrix):
    """Symmetrically normalize adjacency matrix."""
    row_sum = np.array(adj_matrix.sum(1))
    d_inv_sqrt = np.power(row_sum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)

    normalized_matrix = adj_matrix.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt)

    return normalized_matrix.tocsr()

def remove_diag(adj_matrix):
    new_adj_matrix = sp.csr_matrix(adj_matrix)
    new_adj_matrix.setdiag(0.0)
    new_adj_matrix.eliminate_zeros()
    return new_adj_matrix

In [2]:
# Model hyper-parameters
tf.flags.DEFINE_string("data_dir", "./data_dir", "The input data directory (default: None)")
tf.flags.DEFINE_integer("nb_hop", 1, "The order of the real adjacency matrix (default:1)")
tf.flags.DEFINE_string("df_name", "Dunnhumby", "The dataset Name") # 資料集名稱，用來當資料夾名
tf.flags.DEFINE_string("f", "", "kernel")

config = tf.flags.FLAGS
print("---------------------------------------------------")
print("Data_dir = " + str(config.data_dir))
print("\nParameters: " + str(config.__len__()))
for iterVal in config.__iter__():
    print(" + {}={}".format(iterVal, config.__getattr__(iterVal)))
print("Tensorflow version: ", tf.__version__)
print("---------------------------------------------------")

SEED_VALUES = [2, 9, 30, 44, 50, 55, 58, 79, 85, 92]

# ----------------------- MAIN PROGRAM -----------------------
data_dir = config.data_dir
output_dir = data_dir + "/adj_matrix"

df_name = config.df_name

training_file = data_dir + f"/{df_name}_train.txt"
validate_file = data_dir + f"/{df_name}_validate.txt"
test_file = data_dir + f"/{df_name}_test.txt"

print("***************************************************************************************")
print("Output Dir: " + output_dir)

print("@Create output directory")
create_folder(output_dir)

# Load train, validate & test
print("@Load train,validate&test data")
training_instances = read_file_as_lines(training_file)
nb_train = len(training_instances)
print(" + Total training sequences: ", nb_train)

validate_instances = read_file_as_lines(validate_file)
nb_validate = len(validate_instances)
print(" + Total validating sequences: ", nb_validate)

testing_instances = read_file_as_lines(test_file)
nb_test = len(testing_instances)
print(" + Total testing sequences: ", nb_test)

# Create dictionary
print("@Build knowledge")
MAX_SEQ_LENGTH, item_dict, reversed_item_dict, _ = build_knowledge(training_instances, validate_instances, testing_instances)

print("#Statistic")
NB_ITEMS = len(item_dict)
print(" + Maximum sequence length: ", MAX_SEQ_LENGTH)
print(" + Total items: ", NB_ITEMS)

rmatrix_fpath = output_dir +f"/{df_name}/r_matrix_{str(config.nb_hop)}w.npz"

print("@Build the real adjacency matrix")
real_adj_matrix = build_sparse_adjacency_matrix_v2(training_instances, validate_instances, item_dict)
real_adj_matrix = normalize_adj(real_adj_matrix)

mul = real_adj_matrix
with tf.device('/cpu:0'):
    w_mul = real_adj_matrix
    coeff = 1.0
    for w in range(1, config.nb_hop):
        coeff *= 0.85
        w_mul *= real_adj_matrix
        w_mul = remove_diag(w_mul)

        w_adj_matrix = normalize_adj(w_mul)
        mul += coeff * w_adj_matrix

    real_adj_matrix = mul
    if not os.path.exists(output_dir + f"/{df_name}"):
        os.makedirs(output_dir +f"/{df_name}")
    sp.save_npz(rmatrix_fpath, real_adj_matrix)
    print(" + Save adj_matrix to" + rmatrix_fpath)

---------------------------------------------------
Data_dir = ./data_dir

Parameters: 27
 + logtostderr=False
 + alsologtostderr=False
 + log_dir=
 + v=-1
 + verbosity=-1
 + logger_levels={}
 + stderrthreshold=fatal
 + showprefixforinfo=True
 + run_with_pdb=False
 + pdb_post_mortem=False
 + pdb=False
 + run_with_profiling=False
 + profile_file=None
 + use_cprofile_for_profiling=True
 + only_check_args=False
 + op_conversion_fallback_to_while_loop=True
 + runtime_oom_exit=True
 + hbm_oom_exit=True
 + test_srcdir=
 + test_tmpdir=C:\Users\YJLab\AppData\Local\Temp\absl_testing
 + test_random_seed=301
 + test_randomize_ordering_seed=
 + xml_output_file=
 + data_dir=./data_dir
 + nb_hop=1
 + df_name=Dunnhumby
 + f=C:\Users\YJLab\AppData\Roaming\jupyter\runtime\kernel-8bbf1a77-74c4-4875-80b2-6580866facf1.json
Tensorflow version:  2.10.0
---------------------------------------------------
***************************************************************************************
Output Dir: ./dat

# layers

In [3]:
def create_rnn_cell(cell_type, state_size, default_initializer, reuse=None):
    if cell_type == 'GRU':
        return tf.nn.rnn_cell.GRUCell(state_size, activation=tf.nn.tanh, reuse=reuse)
    elif cell_type == 'LSTM':
        return tf.nn.rnn_cell.LSTMCell(state_size, initializer=default_initializer, activation=tf.nn.tanh, reuse=reuse)
    else:
        return tf.nn.rnn_cell.BasicRNNCell(state_size, activation=tf.nn.tanh, reuse=reuse)


def create_rnn_encoder(x, rnn_units, dropout_rate, seq_length, rnn_cell_type, param_initializer, seed, reuse=None):
    with tf.variable_scope("RNN_Encoder", reuse=reuse):
        rnn_cell = create_rnn_cell(rnn_cell_type, rnn_units, param_initializer)
        rnn_cell = tf.nn.rnn_cell.DropoutWrapper(rnn_cell, input_keep_prob=1 - dropout_rate, seed=seed)
        init_state = rnn_cell.zero_state(tf.shape(x)[0], tf.float32)
        # RNN Encoder: Iteratively compute output of recurrent network
        rnn_outputs, _ = tf.nn.dynamic_rnn(rnn_cell, x, initial_state=init_state, sequence_length=seq_length, dtype=tf.float32)
        return rnn_outputs

def create_basket_encoder(x, dense_units, param_initializer, activation_func=None, name="Basket_Encoder", reuse=None):
    with tf.variable_scope(name, reuse=reuse):
        return tf.layers.dense(x, dense_units, kernel_initializer=param_initializer,
                            bias_initializer=tf.zeros_initializer, activation=activation_func)

def get_last_right_output(full_output, max_length, actual_length, rnn_units):
    batch_size = tf.shape(full_output)[0]
    # Start indices for each sample
    index = tf.range(0, batch_size) * max_length + (actual_length - 1)
    # Indexing
    return tf.gather(tf.reshape(full_output, [-1, rnn_units]), index)

# models

In [4]:
class Model:

    def __init__(self, sess, seed, learning_rate, name='model'):
        self.scope = name
        self.session = sess
        self.seed = seed
        self.learning_rate = tf.constant(learning_rate)

    @abstractmethod
    def train_batch(self, s, s_length, y):
        pass

    @abstractmethod
    def validate_batch(self, s, s_length, y):
        pass

    @abstractmethod
    def generate_prediction(self, s, s_length):
        pass

class Beacon(Model):

    def __init__(self, sess, emb_dim, rnn_units, alpha, 
                 max_seq_length, item_probs, adj_matrix, top_k,
                 batch_size, rnn_cell_type, rnn_dropout_rate, seed, learning_rate):

        super().__init__(sess, seed, learning_rate, name="GRN")

        self.emb_dim = emb_dim
        self.rnn_units = rnn_units

        self.max_seq_length = max_seq_length
        self.nb_items = len(item_probs)
        self.item_probs = item_probs
        self.alpha = alpha
        self.batch_size = batch_size
        self.top_k = top_k

        with tf.variable_scope(self.scope):
            # Initialized for n_hop adjacency matrix
            self.A = tf.Variable (tf.random_uniform (adj_matrix.todense().shape, minval=-0.1, maxval=0.1), trainable=False)

            uniform_initializer = np.ones(shape=(self.nb_items), dtype=np.float32) / self.nb_items
            self.I_B = tf.get_variable(dtype=tf.float32, initializer=tf.constant(uniform_initializer, dtype=tf.float32), name="I_B")
            self.I_B_Diag = tf.nn.relu(tf.diag(self.I_B, name="I_B_Diag"))

            self.C_Basket = tf.get_variable(dtype=tf.float32, initializer=tf.constant(adj_matrix.mean()), name="C_B")
            self.y = tf.placeholder(dtype=tf.float32, shape=(batch_size, self.nb_items), name='Target_basket')      

            # Basket Sequence encoder
            with tf.name_scope("Basket_Sequence_Encoder"):
                self.bseq = tf.sparse_placeholder(shape=(batch_size, self.max_seq_length, self.nb_items), dtype=tf.float32, name="bseq_input")
                self.bseq_length = tf.placeholder(dtype=tf.int32, shape=(batch_size,), name='bseq_length')

                self.bseq_encoder = tf.sparse_reshape(self.bseq, shape=[-1, self.nb_items], name="bseq_2d")
                self.bseq_encoder = self.encode_basket_graph(self.bseq_encoder, self.C_Basket, True)
                self.bseq_encoder = tf.reshape(self.bseq_encoder, shape=[-1, self.max_seq_length, self.nb_items], name="bsxMxN")
                self.bseq_encoder = create_basket_encoder(self.bseq_encoder, emb_dim, param_initializer=tf.initializers.he_uniform(), activation_func=tf.nn.relu)       

                # batch_size x max_seq_length x H
                rnn_encoder = create_rnn_encoder(self.bseq_encoder, self.rnn_units, rnn_dropout_rate, self.bseq_length, rnn_cell_type, 
                                                    param_initializer=tf.initializers.glorot_uniform(), seed=self.seed)
                
                # Hack to build the indexing and retrieve the right output. # batch_size x H
                h_T = get_last_right_output(rnn_encoder, self.max_seq_length, self.bseq_length, self.rnn_units)

            # Next basket estimation
            with tf.name_scope("Next_Basket"):
                W_H = tf.get_variable(dtype=tf.float32, initializer=tf.initializers.glorot_uniform(), shape=(self.rnn_units, self.nb_items), name="W_H")
                
                next_item_probs = tf.nn.sigmoid(tf.matmul(h_T, W_H))
                logits = (1.0 - self.alpha) * next_item_probs + self.alpha * self.encode_basket_graph(next_item_probs, tf.constant(0.0))

            with tf.name_scope("Loss"):
                self.loss = self.compute_loss(logits, self.y)

                self.predictions = tf.nn.sigmoid(logits)
                self.top_k_values, self.top_k_indices = tf.nn.top_k(self.predictions, 200)
                self.f1_at_k, self.recall_at_k, self.precision_at_k = self.compute_F1score_at_topk(top_k)
                self.hr_at_k = self.compute_HR_at_topk(top_k)
                self.ndcg_at_k = self.compute_ndcg_at_topk(top_k)
                self.mae_at_k = self.compute_mae_at_topk(top_k)

                self.f1_at_5, self.recall_at_5, self.precision_at_5  = self.compute_F1score_at_topk(5)
                self.hr_at_5 = self.compute_HR_at_topk(5)
                self.ndcg_at_5 = self.compute_ndcg_at_topk(5)
                self.mae_at_5 = self.compute_mae_at_topk(5)
                
                self.f1_at_10, self.recall_at_10, self.precision_at_10 = self.compute_F1score_at_topk(10)
                self.hr_at_10 = self.compute_HR_at_topk(10)
                self.ndcg_at_10 = self.compute_ndcg_at_topk(10)
                self.mae_at_10 = self.compute_mae_at_topk(10)

                self.f1_at_30, self.recall_at_30, self.precision_at_30 = self.compute_F1score_at_topk(30)
                self.hr_at_30 = self.compute_HR_at_topk(30)
                self.ndcg_at_30 = self.compute_ndcg_at_topk(30)
                self.mae_at_30 = self.compute_mae_at_topk(30)

                self.f1_at_50, self.recall_at_50, self.precision_at_50 = self.compute_F1score_at_topk(50)
                self.hr_at_50 = self.compute_HR_at_topk(50)
                self.ndcg_at_50 = self.compute_ndcg_at_topk(50)
                self.mae_at_50 = self.compute_mae_at_topk(50)
                
                self.f1_at_65, self.recall_at_65, self.precision_at_65 = self.compute_F1score_at_topk(65)
                self.hr_at_65 = self.compute_HR_at_topk(65)
                self.ndcg_at_65 = self.compute_ndcg_at_topk(65)
                self.mae_at_65 = self.compute_mae_at_topk(65)

                self.mrr = tf.convert_to_tensor(0.0)
                self.map = tf.convert_to_tensor(0.0)
                self.pred_and_y = self.return_pred_y()

            # Adam optimizer
            train_op = tf.train.RMSPropOptimizer(learning_rate=self.learning_rate)
            # Op to calculate every variable gradient
            self.grads = train_op.compute_gradients(self.loss, tf.trainable_variables())
            self.update_grads = train_op.apply_gradients(self.grads)

            # Summarize all variables and their gradients
            total_parameters = 0
            print("-------------------- SUMMARY ----------------------")
            tf.summary.scalar("C_Basket", self.C_Basket)

            for grad, var in self.grads:
                tf.summary.histogram(var.name, var)
                tf.summary.histogram(var.name + '/grad', grad)

                shape = var.get_shape()
                variable_parameters = 1
                for dim in shape:
                    variable_parameters *= dim
#                     variable_parameters *= dim.value
                print("+ {:<64} {:<10,} parameter(s)".format(var.name, variable_parameters))
                total_parameters += variable_parameters

            print("Total number of parameters: {:,}".format(total_parameters))
            print("----------------- END SUMMARY ----------------------\n")

            # Create a summary to monitor cost tensor
            tf.summary.scalar("Train_Batch_Loss", self.loss)

            # Create a summary to monitor cost tensor
            tf.summary.scalar("Val_Batch_Loss", self.loss, collections=['validation'])

            # Merge all summaries into a single op
            self.merged_summary_op = tf.summary.merge_all()
            self.val_merged_summary_op = tf.summary.merge_all(key='validation')

    def train_batch(self, s, s_length, y):
        bseq_indices, bseq_values = self.get_sparse_tensor_info(s, True)
        
        _, loss, recall, precision, f1_score, ndcg, mae, recall5, precision5, f1_score5, ndcg5, mae5, \
        recall10, precision10, f1_score10, ndcg10, mae10, recall30, precision30, f1_score30, ndcg30, mae30, \
        recall50, precision50, f1_score50, ndcg50, mae50, recall65, precision65, f1_score65, ndcg65, mae65, \
        summary, pred_and_y = self.session.run(
            [self.update_grads, self.loss, self.recall_at_k, self.precision_at_k, self.f1_at_k, self.ndcg_at_k, self.mae_at_k, 
             self.recall_at_5, self.precision_at_5, self.f1_at_5, self.ndcg_at_5, self.mae_at_5,
             self.recall_at_10, self.precision_at_10, self.f1_at_10, self.ndcg_at_10, self.mae_at_10,
             self.recall_at_30, self.precision_at_30, self.f1_at_30, self.ndcg_at_30, self.mae_at_30,
             self.recall_at_50, self.precision_at_50, self.f1_at_50, self.ndcg_at_50, self.mae_at_50, 
             self.recall_at_65, self.precision_at_65, self.f1_at_65, self.ndcg_at_65, self.mae_at_65, self.merged_summary_op, self.pred_and_y],
            feed_dict={self.bseq_length: s_length, self.y: y,
                       self.bseq.indices: bseq_indices, self.bseq.values: bseq_values})

        (pred,labels) = pred_and_y
        labels = [[j for j, label in enumerate(batch_l) if label == 1] for batch_l in labels]
        # 計算mrr
        mrr = self.calculate_MRR(pred,labels)
        # 計算map
        map = self.calculate_MAP(pred,labels)

        return loss, recall, precision, f1_score, ndcg, mae, recall5, precision5, f1_score5, ndcg5, mae5, recall10, precision10, f1_score10, ndcg10, mae10, recall30, precision30, f1_score30, ndcg30, mae30, recall50, precision50, f1_score50, ndcg50, mae50, recall65, precision65, f1_score65, ndcg65, mae65, mrr, map, summary

    def validate_batch(self, s, s_length, y):
        bseq_indices, bseq_values = self.get_sparse_tensor_info(s, True)

        loss, recall, precision, f1_score, ndcg, mae, recall5, precision5, f1_score5, ndcg5, mae5, \
        recall10, precision10, f1_score10, ndcg10, mae10, recall30, precision30, f1_score30, ndcg30, mae30, \
        recall50, precision50, f1_score50, ndcg50, mae50, recall65, precision65, f1_score65, ndcg65, mae65, \
        summary, pred_and_y = self.session.run(
            [self.loss, self.recall_at_k, self.precision_at_k, self.f1_at_k, self.ndcg_at_k, self.mae_at_k ,
             self.recall_at_5, self.precision_at_5, self.f1_at_5, self.ndcg_at_5, self.mae_at_5,
             self.recall_at_10, self.precision_at_10, self.f1_at_10, self.ndcg_at_10, self.mae_at_10, 
             self.recall_at_30, self.precision_at_30, self.f1_at_30, self.ndcg_at_30, self.mae_at_30,
             self.recall_at_50, self.precision_at_50, self.f1_at_50, self.ndcg_at_50, self.mae_at_50, 
             self.recall_at_65, self.precision_at_65, self.f1_at_65, self.ndcg_at_65, self.mae_at_65, self.val_merged_summary_op, self.pred_and_y],
            feed_dict={ self.bseq_length: s_length, self.y: y,
                        self.bseq.indices: bseq_indices, self.bseq.values: bseq_values})
        
        (pred,labels) = pred_and_y
        labels = [[j for j, label in enumerate(batch_l) if label == 1] for batch_l in labels]
        # 計算mrr
        mrr = self.calculate_MRR(pred,labels)
        # 計算map
        map = self.calculate_MAP(pred,labels)

        return loss, recall, precision, f1_score, ndcg, mae, recall5, precision5, f1_score5, ndcg5, mae5, recall10, precision10, f1_score10, ndcg10, mae10, recall30, precision30, f1_score30, ndcg30, mae30, recall50, precision50, f1_score50, ndcg50, mae50, recall65, precision65, f1_score65, ndcg65, mae65, mrr, map, summary

    def generate_prediction(self, s, s_length, y):
        bseq_indices, bseq_values = self.get_sparse_tensor_info(s, True)
        top_k_values, top_k_indices, recall, precision, f1_score, ndcg, mae, recall5, precision5, f1_score5, ndcg5, mae5, \
        recall10, precision10, f1_score10, ndcg10, mae10, recall30, precision30, f1_score30, ndcg30, mae30, \
        recall50, precision50,f1_score50, ndcg50, mae50, recall65, precision65, f1_score65, ndcg65, mae65, pred_and_y = self.session.run(
            [self.top_k_values, self.top_k_indices, self.recall_at_k, self.precision_at_k, self.f1_at_k, self.ndcg_at_k, self.mae_at_k, 
             self.recall_at_5, self.precision_at_5, self.f1_at_5, self.ndcg_at_5, self.mae_at_5,
             self.recall_at_10, self.precision_at_10, self.f1_at_10, self.ndcg_at_10, self.mae_at_10,
             self.recall_at_30, self.precision_at_30, self.f1_at_30, self.ndcg_at_30, self.mae_at_30,
             self.recall_at_50, self.precision_at_50, self.f1_at_50, self.ndcg_at_50, self.mae_at_50,  
             self.recall_at_65, self.precision_at_65, self.f1_at_65, self.ndcg_at_65, self.mae_at_65, self.pred_and_y],
            feed_dict={self.bseq_length: s_length, self.y: y,
                        self.bseq.indices: bseq_indices, self.bseq.values: bseq_values})
        
        (pred,labels) = pred_and_y
        labels = [[j for j, label in enumerate(batch_l) if label == 1] for batch_l in labels]
        # 計算mrr
        mrr = self.calculate_MRR(pred,labels)
        # 計算map
        map = self.calculate_MAP(pred,labels)

        return top_k_values, top_k_indices, recall, precision, f1_score, ndcg, mae, recall5, precision5, f1_score5, ndcg5, mae5, recall10, precision10, f1_score10, ndcg10, mae10, recall30, precision30, f1_score30, ndcg30, mae30, recall50, precision50, f1_score50, ndcg50, mae50, recall65, precision65, f1_score65, ndcg65, mae65, mrr, map

    def encode_basket_graph(self, binput, beta, is_sparse=False):
        with tf.name_scope("Graph_Encoder"):
            if is_sparse:
                encoder = tf.sparse_tensor_dense_matmul(binput, self.I_B_Diag, name="XxI_B")  
                encoder += self.relu_with_threshold(tf.sparse_tensor_dense_matmul(binput, self.A, name="XxA"), beta)  
            else:
                encoder = tf.matmul(binput, self.I_B_Diag, name="XxI_B")
                encoder += self.relu_with_threshold(tf.matmul(binput, self.A, name="XxA"), beta) 
        return encoder

    def get_item_bias(self):
        return self.session.run(self.I_B)

    def get_sparse_tensor_info(self, x, is_bseq=False):
        indices = []
        if is_bseq:
            for sid, bseq in enumerate(x):
                for t, basket in enumerate(bseq):
                    for item_id in basket:
                        indices.append([sid, t, item_id]) # 第幾個batch的第幾個購物籃的項目ID
        else:
            for bid, basket in enumerate(x):
                for item_id in basket:
                    indices.append([bid, item_id])

        values = np.ones(len(indices), dtype=np.float32)
        indices = np.array(indices, dtype=np.int32)

        return indices, values

    def compute_loss(self, logits, y):
        sigmoid_logits = tf.nn.sigmoid(logits)

        neg_y = (1.0 - y)
        pos_logits = y * logits

        pos_max = tf.reduce_max(pos_logits, axis=1)
        pos_max = tf.expand_dims(pos_max, axis=-1)

        pos_min = tf.reduce_min(pos_logits + neg_y * pos_max, axis=1)
        pos_min = tf.expand_dims(pos_min, axis=-1)

        nb_pos, nb_neg = tf.count_nonzero(y, axis=1), tf.count_nonzero(neg_y, axis=1)
        ratio = tf.cast(nb_neg, dtype=tf.float32) / tf.cast(nb_pos, dtype=tf.float32)

        pos_weight = tf.expand_dims(ratio, axis=-1)
        loss = y * -tf.log(sigmoid_logits) * pos_weight + neg_y * -tf.log(1.0 - tf.nn.sigmoid(logits - pos_min))

        return tf.reduce_mean(loss + 1e-8)
    
    def compute_recall_at_topk(self, k=10):
        top_k_preds = self.get_topk_tensor(self.predictions, k)
        correct_preds = tf.count_nonzero(tf.multiply(self.y, top_k_preds), axis=1) # TP
        actual_bsize = tf.count_nonzero(self.y, axis=1) # TP+FN
        return tf.reduce_mean(tf.cast(correct_preds, dtype=tf.float32) / tf.cast(actual_bsize, dtype=tf.float32))
    


    def return_pred_y(self):
        predictions = tf.convert_to_tensor(self.predictions)
        labels = tf.convert_to_tensor(self.y)
        return (predictions,labels)

    #MRR
    def calculate_MRR(self,predictions, labels_list):
        reciprocal_ranks = []
        # 將預測機率矩陣轉換為 numpy 數組。
        predictions = np.array(predictions, dtype=np.float32)
        # 預先排序預測結果（降序）
        sorted_predictions = np.argsort(predictions)[:,::-1]
        # 遍歷每個查詢的預測結果與目標結果
        for pred, targets in zip(sorted_predictions, labels_list):
            # 將目標結果轉換為集合
            target_set = set(targets)
            # 尋找目標的排名
            rank = next((i + 1 for i, p in enumerate(pred) if p in target_set), 0)
            # 計算倒數排名
            reciprocal_rank = 1 / rank if rank > 0 else 0
            reciprocal_ranks.append(reciprocal_rank)
        # 計算 MRR
        mrr_score = sum(reciprocal_ranks) / len(reciprocal_ranks)
        return mrr_score
    

    #MAP
    def calculate_MAP(self, predictions, labels_list):
        average_precisions = []
        # 將預測機率矩陣轉換為 numpy 數組。
        predictions = np.array(predictions, dtype=np.float32)
        # 預先排序預測結果（降序）
        sorted_predictions = np.argsort(predictions)[:,::-1]
        # 遍歷每個查詢的預測結果與目標結果
        for pred, targets in zip(sorted_predictions, labels_list):
            # 將目標結果轉換為集合
            target_set = set(targets)
            # 計算查詢的精確度
            precision = []
            hits = 0
            for i, p in enumerate(pred):
                if p in target_set:
                    hits += 1
                    precision.append(hits / (i + 1))
            # 計算平均精確度
            if precision:
                average_precision = sum(precision) / len(precision)
                average_precisions.append(average_precision)
        # 計算 MAP
        map_score = sum(average_precisions) / len(average_precisions)
        return map_score
    

    def compute_F1score_at_topk(self, k=10):
        top_k_preds = self.get_topk_tensor(self.predictions, k)
        true_positives = tf.reduce_sum(tf.multiply(self.y, top_k_preds), axis=1)
        predicted_positives = tf.reduce_sum(top_k_preds, axis=1)
        actual_positives = tf.reduce_sum(self.y, axis=1)
        precision = true_positives / predicted_positives
        recall = true_positives / actual_positives
        f1_score = 2 * precision * recall / (precision + recall + 1e-8)  # add epsilon to avoid division by zero
        return tf.reduce_mean(f1_score), tf.reduce_mean(recall), tf.reduce_mean(precision)
    
    def compute_HR_at_topk(self, k=10):
        top_k_preds = self.get_topk_tensor(self.predictions, k)
        hits = tf.reduce_sum(tf.multiply(self.y, top_k_preds), axis=1)
        gt = tf.reduce_sum(self.y, axis=1)
        hr = tf.divide(hits, gt)
        return tf.reduce_mean(hr)

    def compute_ndcg_at_topk(self, k=10):
        NDCG = tf.map_fn(lambda idx: self.get_ndcg(idx, k), tf.cast(tf.range(self.predictions.shape[0]), tf.int64), dtype=tf.float32)
        ndcg = tf.reduce_mean(NDCG)
        return ndcg
    
    def get_ndcg(self, idx, k):
        target_topi = tf.argsort(self.predictions[idx], direction='DESCENDING')[:k]
        ndcg = self.compute_NDCG1(tf.cast(self.y[idx], tf.float32), target_topi, k)
        return ndcg

    def compute_NDCG1(self, groundtruth, pred_rank_list, k):
        pred_rank_list_k = pred_rank_list[:k]
        mask = tf.gather(groundtruth, pred_rank_list_k)
        dcg = tf.reduce_sum(mask /  (tf.math.log(tf.range(2, k+1+1, dtype=tf.float32))/tf.math.log(2.) ) )
        num_real_item = tf.reduce_sum(groundtruth)
        num_item = tf.cast(num_real_item, tf.int32)
        def f1(): return tf.reduce_sum( tf.ones(num_item, dtype=tf.float32) / ((tf.math.log(tf.cast(tf.range(2, k+1+1), tf.float32))/tf.math.log(2.) )[:num_item]) )
        def f2(): return tf.reduce_sum( tf.ones(k, dtype=tf.float32) / (tf.math.log(tf.cast(tf.range(2, k+1+1), tf.float32))/tf.math.log(2.) ) )
        idcg = tf.cond( tf.less(num_item,k), f1, f2)
        ndcg = dcg / idcg
        return ndcg
    
    def compute_mae_at_topk(self, k=10):
        MAE = tf.map_fn(lambda idx: self.get_mae(idx, k), tf.cast(tf.range(self.predictions.shape[0]), tf.int64), dtype=tf.float32)
        mae = tf.reduce_mean(MAE)
        return mae
    
    def get_mae(self, idx, k):
        num_real_item = tf.reduce_sum(tf.cast(self.y[idx], tf.float32))
        mae = abs(k-num_real_item)
        return mae
        
        
    # x -> shape(batch_size,N)
    def get_topk_tensor(self, x, k=10):
        _, index_cols = tf.nn.top_k(x, k)

        index_rows = tf.ones(shape=(self.batch_size, k), dtype=tf.int32) * tf.expand_dims(tf.range(0, self.batch_size), axis=-1)

        index_rows = tf.cast(tf.reshape(index_rows, shape=[-1]), dtype=tf.int64)
        index_cols = tf.cast(tf.reshape(index_cols, shape=[-1]), dtype=tf.int64)

        top_k_indices = tf.stack([index_rows, index_cols], axis=1)
        top_k_values = tf.ones(shape=[self.batch_size * k], dtype=tf.float32)

        sparse_tensor = tf.SparseTensor(indices=top_k_indices, values=top_k_values, dense_shape=[self.batch_size, self.nb_items])
        return tf.sparse_tensor_to_dense(tf.sparse_reorder(sparse_tensor))
    
    def relu_with_threshold(self, x, threshold):
        return tf.nn.relu(x - tf.abs(threshold))
    

# procedure

In [5]:
def train_network(sess, net, train_generator, validate_generator, nb_epoch, 
                  total_train_batches, total_validate_batches, display_step,
                  early_stopping_k, epsilon, tensorboard_dir, output_dir,topN_dir,
                  test_generator, total_test_batches,config):
    summary_writer = None
    if tensorboard_dir is not None:
        summary_writer = tf.summary.FileWriter(tensorboard_dir)
    # Add ops to save and restore all the variables.
    saver = tf.train.Saver()

    val_best_performance = [sys.float_info.max]
    patience_cnt = 0
    print("nb_epoch=",nb_epoch," ,early_stopping_k=",early_stopping_k)
    results = []
    for epoch in range(0, nb_epoch):
        print("\n=========================================")
        print("@Epoch#" + str(epoch))

        train_loss = 0.0
        train_f1_score = 0.0
        train_recall = 0.0
        train_precision = 0.0
        train_ndcg = 0.0
        train_mae = 0.0
        # Top5
        train_f1_score5 = 0.0
        train_recall5 = 0.0
        train_precision5 = 0.0
        train_ndcg5 = 0.0
        train_mae5 = 0.0
        # Top10
        train_f1_score10 = 0.0
        train_recall10 = 0.0
        train_precision10 = 0.0
        train_ndcg10 = 0.0
        train_mae10 = 0.0
        # Top30
        train_f1_score30 = 0.0
        train_recall30 = 0.0
        train_precision30 = 0.0
        train_ndcg30 = 0.0
        train_mae30 = 0.0
        # Top50
        train_f1_score50 = 0.0
        train_recall50 = 0.0
        train_precision50 = 0.0
        train_ndcg50 = 0.0
        train_mae50 = 0.0
        # Top65
        train_f1_score65 = 0.0
        train_recall65 = 0.0
        train_precision65 = 0.0
        train_ndcg65 = 0.0
        train_mae65 = 0.0
        
        for batch_id, data in train_generator:
            start_time = time.time()

            loss, recall, precision, f1_score, ndcg, mae, recall5, precision5, f1_score5, ndcg5, mae5, recall10, precision10, f1_score10, ndcg10, mae10, \
            recall30, precision30, f1_score30, ndcg30, mae30, recall50, precision50, f1_score50, ndcg50, mae50, recall65, precision65, f1_score65, ndcg65, mae65, mrr, map, summary  = net.train_batch(data['S'], data['L'], data['Y'])

            train_loss += loss
            avg_train_loss = train_loss / (batch_id + 1)
            
            train_recall += recall
            avg_train_recall = train_recall / (batch_id + 1)
            train_precision += precision
            avg_train_precision = train_precision / (batch_id + 1)
            train_f1_score += f1_score
            avg_train_f1_score = train_f1_score / (batch_id + 1)
            train_ndcg += ndcg
            avg_train_ndcg = train_ndcg / (batch_id + 1)
            train_mae += mae
            avg_train_mae = train_mae / (batch_id+1)
            # Top5
            train_recall5 += recall5
            train_precision5 += precision5
            train_f1_score5 += f1_score5
            train_ndcg5 += ndcg5
            train_mae5 += train_mae5
            # Top10
            train_recall10 += recall10
            train_precision10 += precision10
            train_f1_score10 += f1_score10
            train_ndcg10 += ndcg10
            train_mae10 += train_mae10
            # Top30
            train_recall30 += recall30
            train_precision30 += precision30
            train_f1_score30 += f1_score30
            train_ndcg30 += ndcg30
            train_mae30 += train_mae30
            # Top50
            train_recall50 += recall50
            train_precision50 += precision50
            train_f1_score50 += f1_score50
            train_ndcg50 += ndcg50
            train_mae50 += train_mae50
            # Top65
            train_recall65 += recall65
            train_precision65 += precision65
            train_f1_score65 += f1_score65
            train_ndcg65 += ndcg65
            train_mae65 += train_mae65
            
            # Write logs at every iteration
            if summary_writer is not None:
                summary_writer.add_summary(summary, epoch * total_train_batches + batch_id)

                loss_sum = tf.Summary()
                loss_sum.value.add(tag="Losses/Train_Loss", simple_value=avg_train_loss)
                summary_writer.add_summary(loss_sum, epoch * total_train_batches + batch_id)

                f1_score_sum = tf.Summary()
                f1_score_sum.value.add(tag="f1_scores/Train_f1_score", simple_value=avg_train_f1_score)
                summary_writer.add_summary(f1_score_sum, epoch * total_train_batches + batch_id)

                ndcg_sum = tf.Summary()
                ndcg_sum.value.add(tag="NDCG/train_NDCG", simple_value=avg_train_ndcg)
                summary_writer.add_summary(ndcg_sum, epoch * total_train_batches + batch_id)


            if batch_id % display_step == 0 or batch_id == total_train_batches - 1:
                running_time = time.time() - start_time
                print("Training | Epoch " + str(epoch) + " | " + str(batch_id + 1) + "/" + str(total_train_batches) 
                    + " | Loss= " + "{:.8f}".format(avg_train_loss)
                    + " | Recall@" + str(net.top_k) + " = " + "{:.8f}".format(avg_train_recall)
                    + " | Precision@" + str(net.top_k) + " = " + "{:.8f}".format(avg_train_precision) 
                    + " | F1_score@"+ str(net.top_k) + " = " + "{:.8f}".format(avg_train_f1_score)  
                    + " | NDCG@"+ str(net.top_k) + " = " + "{:.8f}".format(avg_train_ndcg)
                    + " | MAE@"+ str(net.top_k) + " = " + "{:.8f}".format(avg_train_mae) 
                    + " | Time={:.2f}".format(running_time) + "s")

            if batch_id >= total_train_batches - 1:
                break
        # Top5
        avg_train_recall5 = train_recall5 / (batch_id + 1)
        avg_train_precision5 = train_precision5 / (batch_id + 1)
        avg_train_f1_score5 = train_f1_score5 / (batch_id + 1)
        avg_train_ndcg5 = train_ndcg5 / (batch_id + 1)
        avg_train_mae5 = train_mae5 / (batch_id + 1)
        # Top10
        avg_train_recall10 = train_recall10 / (batch_id + 1)
        avg_train_precision10 = train_precision10 / (batch_id + 1)
        avg_train_f1_score10 = train_f1_score10 / (batch_id + 1)
        avg_train_ndcg10 = train_ndcg10 / (batch_id + 1)
        avg_train_mae10 = train_mae10 / (batch_id + 1)
        # Top30
        avg_train_recall30 = train_recall30 / (batch_id + 1)
        avg_train_precision30 = train_precision30 / (batch_id + 1)
        avg_train_f1_score30 = train_f1_score30 / (batch_id + 1)
        avg_train_ndcg30 = train_ndcg30 / (batch_id + 1)
        avg_train_mae30 = train_mae30 / (batch_id + 1)
        # Top50
        avg_train_recall50 = train_recall50 / (batch_id + 1)
        avg_train_precision50 = train_precision50 / (batch_id + 1)
        avg_train_f1_score50 = train_f1_score50 / (batch_id + 1)
        avg_train_ndcg50 = train_ndcg50 / (batch_id + 1)
        avg_train_mae50 = train_mae50 / (batch_id + 1)
        # Top65
        avg_train_recall65 = train_recall65 / (batch_id + 1)
        avg_train_precision65 = train_precision65 / (batch_id + 1)
        avg_train_f1_score65 = train_f1_score65 / (batch_id + 1)
        avg_train_ndcg65 = train_ndcg65 / (batch_id + 1)
        avg_train_mae65 = train_mae65 / (batch_id + 1)
        
        print("Training | Epoch " + str(epoch) + " | "
            + " | Loss= " + "{:.8f}".format(avg_train_loss)  +"\n"
            + " | Recall@5= " + "{:.8f}".format(avg_train_recall5) + " | Precision@5= " + "{:.8f}".format(avg_train_precision5) + " | F1_score@5= " + "{:.8f}".format(avg_train_f1_score5) + " | NDCG@5= " + "{:.8f}".format(avg_train_ndcg5) + " | MAE@5= " + "{:.8f}".format(avg_train_mae5)+"\n"
            + " | Recall@10= " + "{:.8f}".format(avg_train_recall10) + " | Precision@10= " + "{:.8f}".format(avg_train_precision10) + " | F1_score@10= " + "{:.8f}".format(avg_train_f1_score10) + " | NDCG@10= " + "{:.8f}".format(avg_train_ndcg10) + " | MAE@10= " + "{:.8f}".format(avg_train_mae10)+"\n"
            + " | Recall@30= " + "{:.8f}".format(avg_train_recall30) + " | Precision@30= " + "{:.8f}".format(avg_train_precision30) + " | F1_score@30= " + "{:.8f}".format(avg_train_f1_score30) + " | NDCG@30= " + "{:.8f}".format(avg_train_ndcg30)+ " | MAE@30= " + "{:.8f}".format(avg_train_mae30)+"\n"
            + " | Recall@50= " + "{:.8f}".format(avg_train_recall50) + " | Precision@50= " + "{:.8f}".format(avg_train_precision50) + " | F1_score@50= " + "{:.8f}".format(avg_train_f1_score50) + " | NDCG@50= " + "{:.8f}".format(avg_train_ndcg50)+ " | MAE@50= " + "{:.8f}".format(avg_train_mae50)+"\n" 
            + " | Recall@65= " + "{:.8f}".format(avg_train_recall65) + " | Precision@65= " + "{:.8f}".format(avg_train_precision65) + " | F1_score@65= " + "{:.8f}".format(avg_train_f1_score65) + " | NDCG@65= " + "{:.8f}".format(avg_train_ndcg65)+ " | MAE@65= " + "{:.8f}".format(avg_train_mae65)+"\n" )


        print("\n-------------- VALIDATION LOSS--------------------------")
        val_loss = 0.0
        val_recall = 0.0
        val_precision = 0.0
        val_f1_score = 0.0
        val_ndcg = 0.0
        val_mae = 0.0
        # Top5
        val_recall5 = 0.0
        val_precision5 = 0.0
        val_f1_score5 = 0.0
        val_ndcg5 = 0.0
        val_mae5 = 0.0
        # Top10
        val_recall10 = 0.0
        val_precision10 = 0.0
        val_f1_score10 = 0.0
        val_ndcg10 = 0.0
        val_mae10 = 0.0
        # Top30
        val_recall30 = 0.0
        val_precision30 = 0.0
        val_f1_score30 = 0.0
        val_ndcg30 = 0.0
        val_mae30 = 0.0
        # Top50
        val_recall50 = 0.0
        val_precision50 = 0.0
        val_f1_score50 = 0.0
        val_ndcg50 = 0.0
        val_mae50 = 0.0
        # Top65
        val_recall65 = 0.0
        val_precision65 = 0.0
        val_f1_score65 = 0.0
        val_ndcg65 = 0.0
        val_mae65 = 0.0
        
        val_mrr = 0.0
        val_map = 0.0

        for batch_id, data in validate_generator:
            loss, recall, precision, f1_score, ndcg, mae, recall5, precision5, f1_score5, ndcg5, mae5, recall10, precision10, f1_score10, ndcg10, mae10, recall30, precision30, f1_score30, ndcg30, mae30, recall50, precision50, f1_score50, ndcg50, mae50, recall65, precision65, f1_score65, ndcg65, mae65, mrr, map, summary  = net.validate_batch(data['S'], data['L'], data['Y'])

            val_loss += loss
            avg_val_loss = val_loss / (batch_id + 1)

            val_recall += recall
            avg_val_recall = val_recall / (batch_id + 1)
            val_precision += precision
            avg_val_precision = val_precision / (batch_id + 1)
            val_f1_score += f1_score
            avg_val_f1_score = val_f1_score / (batch_id + 1)
            val_ndcg += ndcg
            avg_val_ndcg = val_ndcg / (batch_id + 1)
            val_mae += mae
            avg_val_mae = val_mae / (batch_id + 1)
            # Top5
            val_recall5 += recall5
            val_precision5 += precision5
            val_f1_score5 += f1_score5
            val_ndcg5 += ndcg5
            val_mae5 += mae5
            # Top10
            val_recall10 += recall10
            val_precision10 += precision10
            val_f1_score10 += f1_score10
            val_ndcg10 += ndcg10
            val_mae10 += mae10
            # Top30
            val_recall30 += recall30
            val_precision30 += precision30
            val_f1_score30 += f1_score30
            val_ndcg30 += ndcg30
            val_mae30 += mae30
            # Top50
            val_recall50 += recall50
            val_precision50 += precision50
            val_f1_score50 += f1_score50
            val_ndcg50 += ndcg50
            val_mae50 += mae50
            # Top65
            val_recall65  += recall65
            val_precision65 += precision65
            val_f1_score65 += f1_score65
            val_ndcg65 += ndcg65
            val_mae65 += mae65
            
            val_mrr += mrr
            avg_val_mrr = val_mrr / (batch_id + 1)
            val_map += map
            avg_val_map = val_map / (batch_id + 1)

            # Write logs at every iteration
            if summary_writer is not None:
                summary_writer.add_summary(summary, epoch * total_validate_batches + batch_id)

                loss_sum = tf.Summary()
                loss_sum.value.add(tag="Losses/Val_Loss", simple_value=avg_val_loss)
                summary_writer.add_summary(loss_sum, epoch * total_validate_batches + batch_id)
                
                recall_sum = tf.Summary()
                recall_sum.value.add(tag="Recall/Val_Recall", simple_value=avg_val_recall)
                summary_writer.add_summary(recall_sum, epoch * total_validate_batches + batch_id)

                precision_sum = tf.Summary()
                precision_sum.value.add(tag="Precision/Val_Precision", simple_value=avg_val_precision)
                summary_writer.add_summary(precision_sum, epoch * total_validate_batches + batch_id)

                f1_score_sum = tf.Summary()
                f1_score_sum.value.add(tag="F1_scores/Val_F1_score", simple_value=avg_val_f1_score)
                summary_writer.add_summary(f1_score_sum, epoch * total_validate_batches + batch_id)

                ndcg_sum = tf.Summary()
                ndcg_sum.value.add(tag="NDCG/Val_NDCG", simple_value=avg_val_ndcg)
                summary_writer.add_summary(ndcg_sum, epoch * total_validate_batches + batch_id)
                
                mae_sum = tf.Summary()
                mae_sum.value.add(tag="MAE/Val_MAE", simple_value=avg_val_mae)
                summary_writer.add_summary(mae_sum, epoch * total_validate_batches + batch_id)

                mrr_sum = tf.Summary()
                mrr_sum.value.add(tag="MRR/Val_MRR", simple_value=avg_val_mrr)
                summary_writer.add_summary(mrr_sum, epoch * total_validate_batches + batch_id)

                map_sum = tf.Summary()
                map_sum.value.add(tag="MAP/Val_MAP", simple_value=avg_val_map)
                summary_writer.add_summary(map_sum, epoch * total_validate_batches + batch_id)

            if batch_id % display_step == 0 or batch_id == total_validate_batches - 1:
                print("Validating | Epoch " + str(epoch) + " | " + str(batch_id + 1) + "/" + str(total_validate_batches) 
                    + " | Loss = " + "{:.8f}".format(avg_val_loss)
                    + " | Recall@"+ str(net.top_k) + " = " + "{:.8f}".format(avg_val_recall)
                    + " | Precision@"+ str(net.top_k) + " = " + "{:.8f}".format(avg_val_precision)
                    + " | F1_score@"+ str(net.top_k) + " = " + "{:.8f}".format(avg_val_f1_score)
                    + " | NDCG@"+ str(net.top_k) + " = " + "{:.8f}".format(avg_val_ndcg) 
                    + " | MAE@"+ str(net.top_k) + " = " + "{:.8f}".format(avg_val_mae) )
            
            if batch_id >= total_validate_batches - 1:
                break
        
        # Top5
        avg_val_recall5 = val_recall5 / total_validate_batches
        avg_val_precision5 = val_precision5 / total_validate_batches
        avg_val_f1_score5 = val_f1_score5 / total_validate_batches
        avg_val_ndcg5 = val_ndcg5 / total_validate_batches
        avg_val_mae5 = val_mae5 / total_validate_batches
        # Top10
        avg_val_recall10 = val_recall10 / total_validate_batches
        avg_val_precision10 = val_precision10 / total_validate_batches
        avg_val_f1_score10 = val_f1_score10 / total_validate_batches
        avg_val_ndcg10 = val_ndcg10 / total_validate_batches
        avg_val_mae10 = val_mae10 / total_validate_batches
        # Top30
        avg_val_recall30 = val_recall30 / total_validate_batches
        avg_val_precision30 = val_precision30 / total_validate_batches
        avg_val_f1_score30 = val_f1_score30 / total_validate_batches
        avg_val_ndcg30 = val_ndcg30 / total_validate_batches
        avg_val_mae30 = val_mae30 / total_validate_batches
        # Top50
        avg_val_recall50 = val_recall50 / total_validate_batches
        avg_val_precision50 = val_precision50 / total_validate_batches
        avg_val_f1_score50 = val_f1_score50 / total_validate_batches
        avg_val_ndcg50 = val_ndcg50 / total_validate_batches
        avg_val_mae50 = val_mae50 / total_validate_batches
        # Top65
        avg_val_recall65 = val_recall65 / total_validate_batches
        avg_val_precision65 = val_precision65 / total_validate_batches
        avg_val_f1_score65 = val_f1_score65 / total_validate_batches
        avg_val_ndcg65 = val_ndcg65 / total_validate_batches
        avg_val_mae65 = val_mae65 / total_validate_batches

        avg_val_mrr = val_mrr / total_validate_batches
        avg_val_map = val_map / total_validate_batches

        print("Validating | Epoch " + str(epoch) + " | "
            + " | Loss= " + "{:.8f}".format(avg_val_loss)  +"\n"
            + " | Recall@5= " + "{:.8f}".format(avg_val_recall5) + " | Precision@5= " + "{:.8f}".format(avg_val_precision5) + " | F1_score@5= " + "{:.8f}".format(avg_val_f1_score5) + " | NDCG@5= " + "{:.8f}".format(avg_val_ndcg5) + " | MAE@5= " + "{:.8f}".format(avg_val_mae5)+"\n"
            + " | Recall@10= " + "{:.8f}".format(avg_val_recall10) + " | Precision@10= " + "{:.8f}".format(avg_val_precision10) + " | F1_score@10= " + "{:.8f}".format(avg_val_f1_score10) + " | NDCG@10= " + "{:.8f}".format(avg_val_ndcg10)+ " | MAE@10= " + "{:.8f}".format(avg_val_mae10)+"\n"
            + " | Recall@30= " + "{:.8f}".format(avg_val_recall30) + " | Precision@30= " + "{:.8f}".format(avg_val_precision30) + " | F1_score@30= " + "{:.8f}".format(avg_val_f1_score30) + " | NDCG@30= " + "{:.8f}".format(avg_val_ndcg30)+ " | MAE@30= " + "{:.8f}".format(avg_val_mae30)+"\n"
            + " | Recall@50= " + "{:.8f}".format(avg_val_recall50) + " | Precision@50= " + "{:.8f}".format(avg_val_precision50) + " | F1_score@50= " + "{:.8f}".format(avg_val_f1_score50) + " | NDCG@50= " + "{:.8f}".format(avg_val_ndcg50)+ " | MAE@50= " + "{:.8f}".format(avg_val_mae50)+"\n"
            + " | Recall@65= " + "{:.8f}".format(avg_val_recall65) + " | Precision@65= " + "{:.8f}".format(avg_val_precision65) + " | F1_score@65= " + "{:.8f}".format(avg_val_f1_score65) + " | NDCG@65= " + "{:.8f}".format(avg_val_ndcg65)+ " | MAE@65= " + "{:.8f}".format(avg_val_mae65)+"\n")


        print("\n-------------- TEST LOSS--------------------------")
        test_loss = 0.0
        test_recall = 0.0
        test_precision = 0.0
        test_f1_score = 0.0
        test_ndcg = 0.0
        test_mae = 0.0
        # Top5
        test_recall5 = 0.0
        test_precision5 = 0.0
        test_f1_score5 = 0.0
        test_ndcg5 = 0.0
        test_mae5 = 0.0
        # Top10
        test_recall10 = 0.0
        test_precision10 = 0.0
        test_f1_score10 = 0.0
        test_ndcg10 = 0.0
        test_mae10 = 0.0
        # Top30
        test_recall30 = 0.0
        test_precision30 = 0.0
        test_f1_score30 = 0.0
        test_ndcg30 = 0.0
        test_mae30 = 0.0
        # Top50
        test_recall50 = 0.0
        test_precision50 = 0.0
        test_f1_score50 = 0.0
        test_ndcg50 = 0.0
        test_mae50 = 0.0
        # Top65
        test_recall65 = 0.0
        test_precision65 = 0.0
        test_f1_score65 = 0.0
        test_ndcg65 = 0.0
        test_mae65 = 0.0

        test_mrr = 0.0
        test_map = 0.0
        
        for batch_id, data in test_generator:
            loss, recall, precision, f1_score, ndcg, mae, recall5, precision5, f1_score5, ndcg5, mae5, recall10, precision10, f1_score10, ndcg10, mae10, recall30, precision30, f1_score30, ndcg30, mae30, recall50, precision50, f1_score50, ndcg50, mae50, recall65, precision65, f1_score65, ndcg65, mae65, mrr, map, summary = net.validate_batch(data['S'], data['L'], data['Y'])

            test_loss += loss
            avg_test_loss = test_loss / (batch_id + 1)

            test_recall += recall
            avg_test_recall = test_recall / (batch_id + 1)
            test_precision += precision
            avg_test_precision = test_precision / (batch_id + 1)
            test_f1_score += f1_score
            avg_test_f1_score = test_f1_score / (batch_id + 1)
            test_ndcg += ndcg
            avg_test_ndcg = test_ndcg / (batch_id + 1)
            test_mae += mae
            avg_test_mae = test_mae / (batch_id + 1)
            # Top5
            test_recall5 += recall5
            test_precision5 += precision5
            test_f1_score5 += f1_score5
            test_ndcg5 += ndcg5
            test_mae5 += mae5
            # Top10
            test_recall10 += recall10
            test_precision10 += precision10
            test_f1_score10 += f1_score10
            test_ndcg10 += ndcg10
            test_mae10 += mae10
            # Top30
            test_recall30 += recall30
            test_precision30 += precision30
            test_f1_score30 += f1_score30
            test_ndcg30 += ndcg30
            test_mae30 += mae30
            # Top50
            test_recall50 += recall50
            test_precision50 += precision50
            test_f1_score50 += f1_score50
            test_ndcg50 += ndcg50
            test_mae50 += mae50
            # Top65
            test_recall65 += recall65
            test_precision65 += precision65
            test_f1_score65 += f1_score65
            test_ndcg65 += ndcg65
            test_mae65 += mae65

            test_mrr += mrr
            avg_test_mrr = test_mrr / (batch_id + 1)
            test_map += map
            avg_test_map = test_map / (batch_id + 1)

            # Write logs at every iteration
            if summary_writer is not None:
                #summary_writer.add_summary(summary, epoch * total_test_batches + batch_id)

                loss_sum = tf.Summary()
                loss_sum.value.add(tag="Losses/Test_Loss", simple_value=avg_test_loss)
                summary_writer.add_summary(loss_sum, epoch * total_test_batches + batch_id)
                
                recall_sum = tf.Summary()
                recall_sum.value.add(tag="Recall/Test_Recall", simple_value=avg_test_recall)
                summary_writer.add_summary(recall_sum, epoch * total_test_batches + batch_id)
                
                precision_sum = tf.Summary()
                precision_sum.value.add(tag="Precision/Test_Precision", simple_value=avg_test_precision)
                summary_writer.add_summary(precision_sum, epoch * total_test_batches + batch_id)

                f1_score_sum = tf.Summary()
                f1_score_sum.value.add(tag="F1_scores/Test_F1_score", simple_value=avg_test_f1_score)
                summary_writer.add_summary(f1_score_sum, epoch * total_test_batches + batch_id)
                
                ndcg_sum = tf.Summary()
                ndcg_sum.value.add(tag="NDCG/Test_NDCG", simple_value=avg_test_ndcg)
                summary_writer.add_summary(ndcg_sum, epoch * total_test_batches + batch_id)
                
                mae_sum = tf.Summary()
                mae_sum.value.add(tag="MAE/Test_MAE", simple_value=avg_test_mae)
                summary_writer.add_summary(mae_sum, epoch * total_test_batches + batch_id)

                mrr_sum = tf.Summary()
                mrr_sum.value.add(tag="MRR/Test_MRR", simple_value=avg_test_mrr)
                summary_writer.add_summary(mrr_sum, epoch * total_test_batches + batch_id)

                map_sum = tf.Summary()
                map_sum.value.add(tag="MAP/Test_MAP", simple_value=avg_test_map)
                summary_writer.add_summary(map_sum, epoch * total_test_batches + batch_id)

            if batch_id % display_step == 0 or batch_id == total_test_batches - 1:
                print("Testing | Epoch " + str(epoch) + " | " + str(batch_id + 1) + "/" + str(total_test_batches) 
                    + " | Loss = " + "{:.8f}".format(avg_test_loss)
                    + " | Recall@"+ str(net.top_k) + " = " + "{:.8f}".format(avg_test_recall)
                    + " | Precision@"+ str(net.top_k) + " = " + "{:.8f}".format(avg_test_precision)
                    + " | F1_score@"+ str(net.top_k) + " = " + "{:.8f}".format(avg_test_f1_score)
                    + " | NDCG@"+ str(net.top_k) + " = " + "{:.8f}".format(avg_test_ndcg)
                    + " | MAE@"+ str(net.top_k) + " = " + "{:.8f}".format(avg_test_mae) )
            
            if batch_id >= total_test_batches - 1:
                break
        
        ### 印出Test的評估指標 ###
        # Top5
        avg_test_recall5 = test_recall5 / total_test_batches
        avg_test_precision5 = test_precision5 / total_test_batches
        avg_test_f1_score5 = test_f1_score5 / total_test_batches
        avg_test_ndcg5 = test_ndcg5 / total_test_batches
        avg_test_mae5 = test_mae5 / total_test_batches
        # Top10
        avg_test_recall10 = test_recall10 / (batch_id + 1)
        avg_test_precision10 = test_precision10 / (batch_id + 1)
        avg_test_f1_score10 = test_f1_score10 / (batch_id + 1)
        avg_test_ndcg10 = test_ndcg10 / (batch_id + 1)
        avg_test_mae10 = test_mae10 / (batch_id + 1)
        # Top30
        avg_test_recall30 = test_recall30 / (batch_id + 1)
        avg_test_precision30 = test_precision30 / (batch_id + 1)
        avg_test_f1_score30 = test_f1_score30 / (batch_id + 1)
        avg_test_ndcg30 = test_ndcg30 / (batch_id + 1)
        avg_test_mae30 = test_mae30 / (batch_id + 1)
        # Top50
        avg_test_recall50 = test_recall50 / (batch_id + 1)
        avg_test_precision50 = test_precision50 / (batch_id + 1)
        avg_test_f1_score50 = test_f1_score50 / (batch_id + 1)
        avg_test_ndcg50 = test_ndcg50 / (batch_id + 1)
        avg_test_mae50 = test_mae50 / (batch_id + 1)
        # Top65
        avg_test_recall65 = test_recall65 / (batch_id + 1)
        avg_test_precision65 = test_precision65 / (batch_id + 1)
        avg_test_f1_score65 = test_f1_score65 / (batch_id + 1)
        avg_test_ndcg65 = test_ndcg65 / (batch_id + 1)
        avg_test_mae65 = test_mae65 / (batch_id + 1)

        avg_test_mrr = test_mrr / (batch_id + 1)
        avg_test_map = test_map / (batch_id + 1)

        print("Testing | Epoch " + str(epoch) + " | "+"\n"
            + " | Valid Loss= " + "{:.8f}".format(avg_val_loss)  +"\n"
            + " | Test Loss= " + "{:.8f}".format(avg_test_loss)  +"\n"
            + " | Recall@5= " + "{:.8f}".format(avg_test_recall5) + " | Precision@5= " + "{:.8f}".format(avg_test_precision5) + " | F1_score@5= " + "{:.8f}".format(avg_test_f1_score5) + " | NDCG@5= " + "{:.8f}".format(avg_test_ndcg5) + " | MAE@5= " + "{:.8f}".format(avg_test_mae5)+"\n"
            + " | Recall@10= " + "{:.8f}".format(avg_test_recall10) + " | Precision@10= " + "{:.8f}".format(avg_test_precision10) + " | F1_score@10= " + "{:.8f}".format(avg_test_f1_score10) + " | NDCG@10= " + "{:.8f}".format(avg_test_ndcg10) + " | MAE@10= " + "{:.8f}".format(avg_test_mae10)+"\n"
            + " | Recall@30= " + "{:.8f}".format(avg_test_recall30) + " | Precision@30= " + "{:.8f}".format(avg_test_precision30) + " | F1_score@30= " + "{:.8f}".format(avg_test_f1_score30) + " | NDCG@30= " + "{:.8f}".format(avg_test_ndcg30) + " | MAE@30= " + "{:.8f}".format(avg_test_mae30)+"\n"
            + " | Recall@50= " + "{:.8f}".format(avg_test_recall50) + " | Precision@50= " + "{:.8f}".format(avg_test_precision50) + " | F1_score@50= " + "{:.8f}".format(avg_test_f1_score50) + " | NDCG@50= " + "{:.8f}".format(avg_test_ndcg50) + " | MAE@50= " + "{:.8f}".format(avg_test_mae50)+"\n"
            + " | Recall@65= " + "{:.8f}".format(avg_test_recall65) + " | Precision@65= " + "{:.8f}".format(avg_test_precision65) + " | F1_score@65= " + "{:.8f}".format(avg_test_f1_score65) + " | NDCG@65= " + "{:.8f}".format(avg_test_ndcg65) + " | MAE@65= " + "{:.8f}".format(avg_test_mae65)+"\n" )

        output_file = topN_dir + f"/topN/prediction_TopK_{config.output_number}.txt"
        f = open(output_file, "a")
        f.write(f"--------------------------Epoch {str(epoch)}--------------------------------------" + "\n")
        f.write(f"Loss: {str(avg_val_loss)}" + "\n")
        f.write("Recall@5: "+ str(avg_test_recall5) + " | " + "Precision@5: "+ str(avg_test_precision5) + " | " + "F1 score@5: "+ str(avg_test_f1_score5)+ " | "+"NDCG@5: "+ str(avg_test_ndcg5) + " | " + "MAE@5: "+ str(avg_test_mae5) + "\n")
        f.write("Recall@10: "+ str(avg_test_recall10) + " | " + "Precision@10: "+ str(avg_test_precision10) + " | " + "F1 score@10: "+ str(avg_test_f1_score10)+ " | "+"NDCG@10: "+ str(avg_test_ndcg10) + " | " + "MAE@10: "+ str(avg_test_mae10) +"\n")
        f.write("Recall@30: "+ str(avg_test_recall30) + " | " + "Precision@30: "+ str(avg_test_precision30) + " | " + "F1 score@30: "+ str(avg_test_f1_score30)+ " | "+"NDCG@30: "+ str(avg_test_ndcg30) + " | " + "MAE@30: "+ str(avg_test_mae30) +"\n")
        f.write("Recall@50: "+ str(avg_test_recall50) + " | " + "Precision@50: "+ str(avg_test_precision50) + " | " + "F1 score@50: "+ str(avg_test_f1_score50)+ " | "+"NDCG@50: "+ str(avg_test_ndcg50) + " | " + "MAE@50: "+ str(avg_test_mae50) +"\n")
        f.write("Recall@65: "+ str(avg_test_recall65) + " | " + "Precision@65: "+ str(avg_test_precision65) + " | " + "F1 score@65: "+ str(avg_test_f1_score65)+ " | "+"NDCG@65: "+ str(avg_test_ndcg65) + " | " + "MAE@65: "+ str(avg_test_mae65) +"\n")
#         f.write("MRR: "+ str(avg_test_mrr) + "\n")
#         f.write("MRR: "+ str(avg_test_mrr) + "\n")
#         f.write("MAP: "+ str(avg_test_map) + "\n")
        f.close()
        print(" ==> PREDICTION HAS BEEN DONE!\n")

        result = [epoch] + [avg_test_recall5] +[avg_test_recall10]+[avg_test_recall30]+[avg_test_recall50]+[avg_test_recall65]+\
              [avg_test_precision5] +[avg_test_precision10]+[avg_test_precision30]+[avg_test_precision50]+[avg_test_precision65]+\
              [avg_test_f1_score5]+ [avg_test_f1_score10]+[avg_test_f1_score30] + [avg_test_f1_score50] +[avg_test_f1_score65] +\
              [avg_test_ndcg5] + [avg_test_ndcg10] + [avg_test_ndcg30] + [avg_test_ndcg50] + [avg_test_ndcg65] +\
              [avg_test_mae5] + [avg_test_mae10] + [avg_test_mae30] + [avg_test_mae50] + [avg_test_mae65] +[avg_val_loss]
        
        results.append(result)

        if summary_writer is not None:
            I_B= net.get_item_bias()
            item_probs = net.item_probs

            I_B_corr = np.corrcoef(I_B, item_probs)
            I_B_summ = tf.Summary()
            I_B_summ.value.add(tag="CorrCoef/Item_Bias", simple_value=I_B_corr[1][0])
            summary_writer.add_summary(I_B_summ, epoch)

        avg_val_loss = val_loss / total_validate_batches
        print("\n@ The validation's loss = " + str(avg_val_loss))
        imprv_ratio = (val_best_performance[-1] - avg_val_loss)/val_best_performance[-1]
        if imprv_ratio > epsilon:
            print("# The validation's loss is improved from " + "{:.8f}".format(val_best_performance[-1]) + \
                  " to " + "{:.8f}".format(avg_val_loss))
            val_best_performance.append(avg_val_loss)

            patience_cnt = 0
        else:
            patience_cnt += 1
        
        if patience_cnt >= early_stopping_k:
            print("# The training is early stopped at Epoch " + str(epoch))
            break
    record_df = pd.DataFrame(results,columns=['Epoch', 'Recall@5', 'Recall@10', 'Recall@30', 'Recall@50', 'Recall@65',
                                              'Precision@5', 'Precision@10', 'Precision@30', 'Precision@50', 'Precision@65',
                                              'F1-score@5', 'F1-score@10', 'F1-score@30', 'F1-score@50', 'F1-score@65',
                                              'NDCG@5', 'NDCG@10', 'NDCG@30', 'NDCG@50', 'NDCG@65', 
                                              'MAE@5', 'MAE@10', 'MAE@30', 'MAE@50', 'MAE@65', 'val_loss'])
    
    path = topN_dir + f'/topN/output_{config.learning_rate}_{config.output_number}.csv'
    record_df.to_csv(path, index=False)

def tune(net, data_generator, total_batches, display_step, output_file):
    f = open(output_file, "w")
    val_loss = 0.0
    val_recall = 0.0
    val_precision = 0.0
    val_f1_score = 0.0
    val_ndcg = 0.0
    val_mae = 0.0
    for batch_id, data in data_generator:
        loss, recall, precision, f1_score, ndcg, mae, recall5, precision5, f1_score5, ndcg5, mae5, recall10, precision10, f1_score10, ndcg10, mae10, recall30, precision30, f1_score30, ndcg30, mae30, recall50, precision50, f1_score50, ndcg50, mae50, recall65, precision65, f1_score65, ndcg65, mae65, _ = net.validate_batch(data['S'], data['L'], data['Y'])

        val_loss += loss
        avg_val_loss = val_loss / (batch_id + 1)

        val_recall += recall
        avg_val_recall = val_recall / (batch_id + 1)
        
        val_precision += precision
        avg_val_precision = val_precision / (batch_id + 1)
        
        val_f1_score += f1_score
        avg_val_f1_score = val_f1_score / (batch_id + 1)

        val_ndcg += ndcg
        avg_val_ndcg = val_ndcg / (batch_id + 1)
        
        val_mae += mae
        avg_val_mae = val_mae / (batch_id + 1)

        # Write logs at every iteration
        if batch_id % display_step == 0 or batch_id == total_batches - 1:
            print(str(batch_id + 1) + "/" + str(total_batches) + " | Loss = " + "{:.8f}".format(avg_val_loss)
                    + " | Recall@"+ str(net.top_k) + " = " + "{:.8f}".format(avg_val_recall)
                    + " | Precision@"+ str(net.top_k) + " = " + "{:.8f}".format(avg_val_precision)
                    + " | F1_score@"+ str(net.top_k) + " = " + "{:.8f}".format(avg_val_f1_score)
                    + " | NDCG@"+ str(net.top_k) + " = " + "{:.8f}".format(avg_val_ndcg)
                    + " | MAE@"+ str(net.top_k) + " = " + "{:.8f}".format(avg_val_mae))

        if batch_id >= total_batches - 1:
            break
        
    avg_val_recall = val_recall / total_batches
    avg_val_precision = val_precision / total_batches
    avg_val_f1_score = val_f1_score / total_batches
    avg_val_ndcg = val_ndcg / total_batches
    avg_val_mae = val_mae / total_batches
    f.write(str(avg_val_recall) + "\n")
    f.write(str(avg_val_precision) + "\n")
    f.write(str(avg_val_f1_score) + "\n")
    f.write(str(avg_val_ndcg) + "\n")
    f.write(str(avg_val_mae) + "\n")
    f.close()


def generate_prediction(net, data_generator, total_test_batches, display_step, inv_item_dict, output_file):
    f = open(output_file, "w")
    val_recall = 0.0
    val_precision = 0.0
    val_f1_score = 0.0
    val_ndcg = 0.0
    val_mae = 0.0
    for batch_id, data in data_generator:
        values, indices, recall, precision, f1_score, ndcg, mae, recall5, precision5, f1_score5, \
        ndcg5, mae5, recall10, precision10, f1_score10, ndcg10, mae10, recall30, precision30, \
        f1_score30, ndcg30, mae30, recall50, precision50, f1_score50, ndcg50, mae50, recall65, precision65, f1_score65, ndcg65, mae65, mrr = net.generate_prediction(data['S'], data['L'], data['Y'])
        
        val_recall += recall
        avg_val_recall = val_recall / (batch_id + 1)
        
        val_precision += precision
        avg_val_precision = val_precision / (batch_id + 1)
        
        val_f1_score += f1_score
        avg_val_f1_score = val_f1_score / (batch_id + 1)

        val_ndcg += ndcg
        avg_val_ndcg = val_ndcg / (batch_id + 1)
        
        val_mae += mae
        avg_val_mae = val_mae / (batch_id + 1)

        for i, (seq_val, seq_ind) in enumerate(zip(values, indices)):
            f.write("Target:" + data['O'][i])

            for (v, idx) in zip(seq_val, seq_ind):
                f.write("|" + str(inv_item_dict[idx]) + ":" + str(v))

            f.write("\n")

        if batch_id % display_step == 0 or batch_id == total_test_batches - 1:
            print(str(batch_id + 1) + "/" + str(total_test_batches))

        if batch_id >= total_test_batches - 1:
            break

    avg_val_recall = val_recall / total_test_batches
    avg_val_precision = val_precision / total_test_batches
    avg_val_f1_score = val_f1_score / total_test_batches
    avg_val_ndcg = val_ndcg / total_test_batches
    avg_val_mae = val_mae / total_test_batches
    f.write("----------------------------------------------------------------" + "\n")
    f.write("Recall@"+ str(net.top_k) +": "+ str(avg_val_recall) + "\n")
    f.write("Precision@"+ str(net.top_k) +": "+ str(avg_val_precision) + "\n")
    f.write("F1 score@"+ str(net.top_k) +": "+ str(avg_val_f1_score) + "\n")
    f.write("NDCG@"+ str(net.top_k) +": "+ str(avg_val_ndcg) + "\n")
    f.write("MAE@"+ str(net.top_k) +": "+ str(avg_val_mae) + "\n")
    f.close()
    print(" ==> PREDICTION HAS BEEN DONE!")


def recent_model_dir(dir):
    folder_list = list_directory(dir, True)
    folder_list = sorted(folder_list, key=get_epoch)
    return folder_list[-1]


def get_epoch(x):
    idx = x.index('_') + 1
    return int(x[idx:])

# main

In [6]:
# Parameters
# ###########################
# GPU & Seed

tf.flags.DEFINE_string("device_id", 'None', "GPU device is to be used in training (default: None)")
tf.flags.DEFINE_integer("seed", 45, "Seed value for reproducibility (default: 89)") # 2、5、10、15、20、25、30、35、40、45

# Model hyper-parameters
tf.flags.DEFINE_string("output_dir", "./output_dir", "The output directory (default: None)")
tf.flags.DEFINE_string("tensorboard_dir", "/tensorboard_dir", "The tensorboard directory (default: None")

tf.flags.DEFINE_integer("emb_dim", 32, "The dimensionality of embedding (default: 2)")
tf.flags.DEFINE_integer("rnn_unit", 4, "The number of hidden units of RNN (default: 4)")
tf.flags.DEFINE_float("alpha", 0.5, "The reguralized hyper-parameter (default: 0.5)")

tf.flags.DEFINE_integer("matrix_type", 1, "The type of adjacency matrix (0=zero,1=real,default:1)")

# Training hyper-parameters
tf.flags.DEFINE_integer("nb_epoch", 35, "Number of epochs (default: 15)")  
tf.flags.DEFINE_integer("early_stopping_k", 35, "Early stopping patience (default: 5)")
tf.flags.DEFINE_float("learning_rate", 0.001, "Learning rate (default: 0.001)")
tf.flags.DEFINE_float("epsilon", 1e-8, "The epsilon threshold in training (default: 1e-8)")
tf.flags.DEFINE_float("dropout_rate", 0.3, "Dropout keep probability for RNN (default: 0.3)")
tf.flags.DEFINE_integer("batch_size", 8, "Batch size (default: 32)")
tf.flags.DEFINE_integer("display_step", 10, "Show loss/acc for every display_step batches (default: 10)")
tf.flags.DEFINE_string("rnn_cell_type", "LSTM", " RNN Cell Type like LSTM, GRU, etc. (default: LSTM)")
tf.flags.DEFINE_integer("top_k", 40, "Top K Accuracy (default: 10)")
tf.flags.DEFINE_boolean("train_mode", True, "Turn on/off the training mode (default: False)")
# tf.flags.DEFINE_boolean("tune_mode", False, "Turn on/off the tunning mode (default: False)")
# tf.flags.DEFINE_boolean("prediction_mode", False, "Turn on/off the testing mode (default: False)")
tf.flags.DEFINE_integer("output_number", 1, "The output number  (default: 1)")

config = tf.flags.FLAGS
print(config.seed)
print("---------------------------------------------------")
print("SeedVal = " + str(config.seed))
print("\nParameters: " + str(config.__len__()))
for iterVal in config.__iter__():
    print(" + {}={}".format(iterVal, config.__getattr__(iterVal)))
print("Tensorflow version: ", tf.__version__)
print("---------------------------------------------------")

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = config.device_id

# for reproducibility
np.random.seed(config.seed)
tf.compat.v1.set_random_seed(config.seed)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

gpu_config = tf.ConfigProto()
gpu_config.gpu_options.allow_growth = True
gpu_config.gpu_options.per_process_gpu_memory_fraction = 0.58
gpu_config.log_device_placement = False

# ----------------------- MAIN PROGRAM -----------------------

df_name = config.df_name

data_dir = config.data_dir
output_dir = config.output_dir +f"/{df_name}"
tensorboard_dir=config.tensorboard_dir

print("df_name=",df_name)
training_file = data_dir + f"/{df_name}_train.txt"
validate_file = data_dir + f"/{df_name}_validate.txt"
testing_file = data_dir + f"/{df_name}_test.txt"

print("***************************************************************************************")
print("Output Dir: " + output_dir)

# Create directories
print("@Create directories")
create_folder(output_dir + "/models")
create_folder(output_dir + "/topN")

if tensorboard_dir is not None:
    create_folder(tensorboard_dir)

# Load train, validate & test
print("@Load train,validate&test data")
training_instances = read_file_as_lines(training_file)
nb_train = len(training_instances)
print(" + Total training sequences: ", nb_train)

validate_instances = read_file_as_lines(validate_file)
nb_validate = len(validate_instances)
print(" + Total validating sequences: ", nb_validate)

testing_instances = read_file_as_lines(testing_file)
nb_test = len(testing_instances)
print(" + Total testing sequences: ", nb_test)

# Create dictionary
print("@Build knowledge")
MAX_SEQ_LENGTH, item_dict, rev_item_dict, item_probs = build_knowledge(training_instances, validate_instances, testing_instances)

print("#Statistic")
NB_ITEMS = len(item_dict)
print(" + Maximum sequence length: ", MAX_SEQ_LENGTH)
print(" + Total items: ", NB_ITEMS)

matrix_type = config.matrix_type
if matrix_type == 0:
    print("@Create an zero adjacency matrix")
    adj_matrix = create_zero_matrix(NB_ITEMS)
else:
    print("@Load the normalized adjacency matrix")
    matrix_fpath = data_dir + f"/adj_matrix/{df_name}/r_matrix_{str(config.nb_hop)}w.npz"
    adj_matrix = sp.load_npz(matrix_fpath)
    print(" + Real adj_matrix has been loaded from" + matrix_fpath)


print("@Compute #batches in train/validation/test")
total_train_batches = compute_total_batches(nb_train, config.batch_size)
total_validate_batches = compute_total_batches(nb_validate, config.batch_size)
total_test_batches = compute_total_batches(nb_test, config.batch_size)
print(" + #batches in train ", total_train_batches)
print(" + #batches in validate ", total_validate_batches)
print(" + #batches in test ", total_test_batches)

model_dir = output_dir + "/models"
if config.train_mode:
    with tf.Session(config=gpu_config) as sess:
        # Training
        # ==================================================
        # Create data generator
        train_generator = seq_batch_generator(training_instances, item_dict, config.batch_size)
        validate_generator = seq_batch_generator(validate_instances, item_dict, config.batch_size, False)
        test_generator = seq_batch_generator(testing_instances, item_dict, config.batch_size, False)
        
        # Initialize the network
        print(" + Initialize the network")
        net = Beacon(sess, config.emb_dim, config.rnn_unit, config.alpha, MAX_SEQ_LENGTH, item_probs, adj_matrix, config.top_k, 
                             config.batch_size, config.rnn_cell_type, config.dropout_rate, config.seed, config.learning_rate)

        print(" + Initialize parameters")
        sess.run(tf.global_variables_initializer())

        print("================== TRAINING ====================")
        print("@Start training")
        train_network(sess, net, train_generator, validate_generator, config.nb_epoch,
                                total_train_batches, total_validate_batches, config.display_step,
                                config.early_stopping_k, config.epsilon, tensorboard_dir, model_dir,output_dir,
                                test_generator, total_test_batches, config)

        # Reset before re-load
    tf.reset_default_graph()

# 執行時，當前目錄須移至"\程式碼\ASBRec_Code\baseline\beacon"
# python main.py

45
---------------------------------------------------
SeedVal = 45

Parameters: 46
 + logtostderr=False
 + alsologtostderr=False
 + log_dir=
 + v=-1
 + verbosity=-1
 + logger_levels={}
 + stderrthreshold=fatal
 + showprefixforinfo=True
 + run_with_pdb=False
 + pdb_post_mortem=False
 + pdb=False
 + run_with_profiling=False
 + profile_file=None
 + use_cprofile_for_profiling=True
 + only_check_args=False
 + op_conversion_fallback_to_while_loop=True
 + runtime_oom_exit=True
 + hbm_oom_exit=True
 + test_srcdir=
 + test_tmpdir=C:\Users\YJLab\AppData\Local\Temp\absl_testing
 + test_random_seed=301
 + test_randomize_ordering_seed=
 + xml_output_file=
 + data_dir=./data_dir
 + nb_hop=1
 + df_name=Dunnhumby
 + f=C:\Users\YJLab\AppData\Roaming\jupyter\runtime\kernel-8bbf1a77-74c4-4875-80b2-6580866facf1.json
 + device_id=None
 + seed=45
 + output_dir=./output_dir
 + tensorboard_dir=/tensorboard_dir
 + emb_dim=32
 + rnn_unit=4
 + alpha=0.5
 + matrix_type=1
 + nb_epoch=35
 + early_stopping_k=35
 + 

C:\Users\YJLab\anaconda3\envs\baseline_env\lib\site-packages\ipykernel_launcher.py:22: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
C:\Users\YJLab\anaconda3\envs\baseline_env\lib\site-packages\ipykernel_launcher.py:5: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  """


-------------------- SUMMARY ----------------------
+ GRN/I_B:0                                                        3,003      parameter(s)
+ GRN/C_B:0                                                        1          parameter(s)
+ GRN/Basket_Encoder/dense/kernel:0                                96,096     parameter(s)
+ GRN/Basket_Encoder/dense/bias:0                                  32         parameter(s)
+ GRN/RNN_Encoder/rnn/lstm_cell/kernel:0                           576        parameter(s)
+ GRN/RNN_Encoder/rnn/lstm_cell/bias:0                             16         parameter(s)
+ GRN/W_H:0                                                        12,012     parameter(s)
Total number of parameters: 111,736
----------------- END SUMMARY ----------------------

 + Initialize parameters
================== TRAINING ====================
@Start training
nb_epoch= 35  ,early_stopping_k= 35

@Epoch#0
Training | Epoch 0 | 1/1123 | Loss= 1.35598838 | Recall@40 = 0.00961538 | Precision@4

Training | Epoch 0 | 391/1123 | Loss= 1.32060744 | Recall@40 = 0.01417888 | Precision@40 = 0.00262948 | F1_score@40 = 0.00393711 | NDCG@40 = 0.00647240 | MAE@40 = 32.15505115 | Time=0.14s
Training | Epoch 0 | 401/1123 | Loss= 1.32074347 | Recall@40 = 0.01438533 | Precision@40 = 0.00264963 | F1_score@40 = 0.00397967 | NDCG@40 = 0.00653998 | MAE@40 = 32.15897756 | Time=0.19s
Training | Epoch 0 | 411/1123 | Loss= 1.32066209 | Recall@40 = 0.01455051 | Precision@40 = 0.00269161 | F1_score@40 = 0.00404358 | NDCG@40 = 0.00661223 | MAE@40 = 32.17122871 | Time=0.24s
Training | Epoch 0 | 421/1123 | Loss= 1.32020550 | Recall@40 = 0.01500082 | Precision@40 = 0.00276128 | F1_score@40 = 0.00414875 | NDCG@40 = 0.00678197 | MAE@40 = 32.16508314 | Time=0.13s
Training | Epoch 0 | 431/1123 | Loss= 1.31994098 | Recall@40 = 0.01513880 | Precision@40 = 0.00280597 | F1_score@40 = 0.00421922 | NDCG@40 = 0.00688453 | MAE@40 = 32.17633411 | Time=0.15s
Training | Epoch 0 | 441/1123 | Loss= 1.31963531 | Recall@40

Training | Epoch 0 | 831/1123 | Loss= 1.30288647 | Recall@40 = 0.02334834 | Precision@40 = 0.00385454 | F1_score@40 = 0.00592782 | NDCG@40 = 0.01327288 | MAE@40 = 32.23315283 | Time=0.28s
Training | Epoch 0 | 841/1123 | Loss= 1.30240525 | Recall@40 = 0.02353974 | Precision@40 = 0.00389046 | F1_score@40 = 0.00597923 | NDCG@40 = 0.01369701 | MAE@40 = 32.21953032 | Time=0.20s
Training | Epoch 0 | 851/1123 | Loss= 1.30217467 | Recall@40 = 0.02376717 | Precision@40 = 0.00393287 | F1_score@40 = 0.00604309 | NDCG@40 = 0.01417125 | MAE@40 = 32.21269095 | Time=0.16s
Training | Epoch 0 | 861/1123 | Loss= 1.30206573 | Recall@40 = 0.02374104 | Precision@40 = 0.00394527 | F1_score@40 = 0.00606367 | NDCG@40 = 0.01438546 | MAE@40 = 32.21370499 | Time=0.16s
Training | Epoch 0 | 871/1123 | Loss= 1.30174590 | Recall@40 = 0.02387242 | Precision@40 = 0.00397532 | F1_score@40 = 0.00611554 | NDCG@40 = 0.01468699 | MAE@40 = 32.21225603 | Time=0.22s
Training | Epoch 0 | 881/1123 | Loss= 1.30112949 | Recall@40

...

Testing | Epoch 33 | 121/323 | Loss = 1.16101129 | Recall@40 = 0.18131522 | Precision@40 = 0.03199897 | F1_score@40 = 0.04918659 | NDCG@40 = 0.14465759 | MAE@40 = 32.32334711
Testing | Epoch 33 | 131/323 | Loss = 1.15924358 | Recall@40 = 0.18374619 | Precision@40 = 0.03272901 | F1_score@40 = 0.05016300 | NDCG@40 = 0.14647003 | MAE@40 = 32.19656489
Testing | Epoch 33 | 141/323 | Loss = 1.16350434 | Recall@40 = 0.18023109 | Precision@40 = 0.03273493 | F1_score@40 = 0.05006565 | NDCG@40 = 0.14411928 | MAE@40 = 32.13741135
Testing | Epoch 33 | 151/323 | Loss = 1.16699972 | Recall@40 = 0.17829532 | Precision@40 = 0.03271937 | F1_score@40 = 0.04996713 | NDCG@40 = 0.14269457 | MAE@40 = 32.13162252
Testing | Epoch 33 | 161/323 | Loss = 1.16794562 | Recall@40 = 0.17602158 | Precision@40 = 0.03251165 | F1_score@40 = 0.04957348 | NDCG@40 = 0.14150442 | MAE@40 = 32.14596273
Testing | Epoch 33 | 171/323 | Loss = 1.16837681 | Recall@40 = 0.17624446 | Precision@40 = 0.03227339 | F1_score@40 = 0.04932

Training | Epoch 34 | 191/1123 | Loss= 1.09332241 | Recall@40 = 0.20378844 | Precision@40 = 0.03412958 | F1_score@40 = 0.05279848 | NDCG@40 = 0.15436643 | MAE@40 = 32.47316754 | Time=0.19s
Training | Epoch 34 | 201/1123 | Loss= 1.08999982 | Recall@40 = 0.20574257 | Precision@40 = 0.03407960 | F1_score@40 = 0.05281741 | NDCG@40 = 0.15431838 | MAE@40 = 32.53669154 | Time=0.16s
Training | Epoch 34 | 211/1123 | Loss= 1.08754165 | Recall@40 = 0.20824515 | Precision@40 = 0.03447867 | F1_score@40 = 0.05334848 | NDCG@40 = 0.15635191 | MAE@40 = 32.48400474 | Time=0.14s
Training | Epoch 34 | 221/1123 | Loss= 1.09103784 | Recall@40 = 0.20705463 | Precision@40 = 0.03458710 | F1_score@40 = 0.05345612 | NDCG@40 = 0.15588140 | MAE@40 = 32.40780543 | Time=0.14s
Training | Epoch 34 | 231/1123 | Loss= 1.08958957 | Recall@40 = 0.20651338 | Precision@40 = 0.03471320 | F1_score@40 = 0.05356289 | NDCG@40 = 0.15577910 | MAE@40 = 32.37391775 | Time=0.14s
Training | Epoch 34 | 241/1123 | Loss= 1.09104414 | Rec

Training | Epoch 34 | 631/1123 | Loss= 1.09126670 | Recall@40 = 0.20201917 | Precision@40 = 0.03544473 | F1_score@40 = 0.05421735 | NDCG@40 = 0.15410339 | MAE@40 = 32.24049128 | Time=0.14s
Training | Epoch 34 | 641/1123 | Loss= 1.09080457 | Recall@40 = 0.20164990 | Precision@40 = 0.03536467 | F1_score@40 = 0.05409905 | NDCG@40 = 0.15395774 | MAE@40 = 32.24570983 | Time=0.23s
Training | Epoch 34 | 651/1123 | Loss= 1.09071506 | Recall@40 = 0.20137821 | Precision@40 = 0.03527266 | F1_score@40 = 0.05398061 | NDCG@40 = 0.15381965 | MAE@40 = 32.24635177 | Time=0.16s
Training | Epoch 34 | 661/1123 | Loss= 1.09066107 | Recall@40 = 0.20096533 | Precision@40 = 0.03524489 | F1_score@40 = 0.05393196 | NDCG@40 = 0.15342449 | MAE@40 = 32.24470499 | Time=0.16s
Training | Epoch 34 | 671/1123 | Loss= 1.09141724 | Recall@40 = 0.20094362 | Precision@40 = 0.03526453 | F1_score@40 = 0.05396284 | NDCG@40 = 0.15339435 | MAE@40 = 32.24292101 | Time=0.14s
Training | Epoch 34 | 681/1123 | Loss= 1.09203417 | Rec

Training | Epoch 34 | 1071/1123 | Loss= 1.09262184 | Recall@40 = 0.19994126 | Precision@40 = 0.03512780 | F1_score@40 = 0.05374063 | NDCG@40 = 0.15314853 | MAE@40 = 32.24953315 | Time=0.19s
Training | Epoch 34 | 1081/1123 | Loss= 1.09213678 | Recall@40 = 0.19980648 | Precision@40 = 0.03503989 | F1_score@40 = 0.05362026 | NDCG@40 = 0.15306168 | MAE@40 = 32.26248844 | Time=0.20s
Training | Epoch 34 | 1091/1123 | Loss= 1.09170468 | Recall@40 = 0.20010363 | Precision@40 = 0.03504812 | F1_score@40 = 0.05363523 | NDCG@40 = 0.15324977 | MAE@40 = 32.26145738 | Time=0.17s
Training | Epoch 34 | 1101/1123 | Loss= 1.09169576 | Recall@40 = 0.20005109 | Precision@40 = 0.03506471 | F1_score@40 = 0.05365355 | NDCG@40 = 0.15334720 | MAE@40 = 32.26328338 | Time=0.21s
Training | Epoch 34 | 1111/1123 | Loss= 1.09135820 | Recall@40 = 0.20054632 | Precision@40 = 0.03514289 | F1_score@40 = 0.05377490 | NDCG@40 = 0.15372668 | MAE@40 = 32.25247525 | Time=0.20s
Training | Epoch 34 | 1121/1123 | Loss= 1.09125749

Testing | Epoch 34 | 151/323 | Loss = 1.16618515 | Recall@40 = 0.18050351 | Precision@40 = 0.03315397 | F1_score@40 = 0.05059924 | NDCG@40 = 0.14400394 | MAE@40 = 32.13162252
Testing | Epoch 34 | 161/323 | Loss = 1.16718212 | Recall@40 = 0.17769420 | Precision@40 = 0.03284162 | F1_score@40 = 0.05004567 | NDCG@40 = 0.14251681 | MAE@40 = 32.14596273
Testing | Epoch 34 | 171/323 | Loss = 1.16733925 | Recall@40 = 0.17781354 | Precision@40 = 0.03262061 | F1_score@40 = 0.04981722 | NDCG@40 = 0.14232405 | MAE@40 = 32.21929825
Testing | Epoch 34 | 181/323 | Loss = 1.16647229 | Recall@40 = 0.18085541 | Precision@40 = 0.03304558 | F1_score@40 = 0.05060624 | NDCG@40 = 0.14559530 | MAE@40 = 32.21477901
Testing | Epoch 34 | 191/323 | Loss = 1.16550704 | Recall@40 = 0.18221261 | Precision@40 = 0.03342605 | F1_score@40 = 0.05116829 | NDCG@40 = 0.14675782 | MAE@40 = 32.17931937
Testing | Epoch 34 | 201/323 | Loss = 1.16775099 | Recall@40 = 0.18080550 | Precision@40 = 0.03328669 | F1_score@40 = 0.05096